# Week 3: Heat Index Calculation
## Implementing the KMA (Korea Meteorological Administration) Heat Index Formula

**Instructor**: Sohn Chul

---

## 🎯 Learning Objectives

By the end of this session, you will be able to:
1. Understand the concept and importance of Heat Index
2. Implement the KMA (Korea Meteorological Administration) Heat Index formula
3. Calculate Stull's wet-bulb temperature for heat index computation
4. Calculate daily, hourly, and monthly heat index values
5. Identify and characterize heatwave events based on heat index thresholds

## 1. Understanding Heat Index

### 1.1 What is Heat Index?

The **Heat Index** (HI), also known as the "apparent temperature" or "feels like" temperature (체감온도), combines air temperature and relative humidity to determine the human-perceived equivalent temperature.

### 1.2 Why is Heat Index Important?

- **Human Health**: Better indicator of heat stress than temperature alone
- **Public Safety**: Used for heat advisories and warnings
- **Urban Planning**: Helps identify areas needing cooling interventions
- **Climate Assessment**: More accurate measure of thermal comfort

### 1.3 KMA Heat Index Categories

| Heat Index (°C) | Category | Health Implications | Korean Term |
|-----------------|----------|---------------------|-------------|
| < 25°C | Comfortable | Normal conditions | 쾌적 |
| 25-28°C | Caution | Fatigue possible with prolonged exposure | 주의 |
| 28-31°C | Extreme Caution | Heat cramps and exhaustion possible | 극도주의 |
| 31-35°C | Danger | Heat exhaustion likely | 위험 |
| > 35°C | Extreme Danger | Heat stroke highly likely | 극도위험 |

### 1.4 KMA Formula Characteristics

The Korea Meteorological Administration (KMA) formula:
- Uses Stull's wet-bulb temperature estimation method
- Provides more accurate results for Korean climate conditions
- Accounts for high humidity typical in Korean summers
- Validated against Korean weather station data

## 2. Setup and Import Libraries

In [ ]:
# Standard libraries
import os
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import pandas as pd
import numpy as np

# Datetime handling
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

# Configure visualization
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Libraries loaded successfully")

## 3. Load Preprocessed Data

In [ ]:
# Load cleaned data from Week 2
DATA_PATH = '../data/processed'
data_file = os.path.join(DATA_PATH, 'sdot_cleaned_data.csv')

# Check if file exists
if os.path.exists(data_file):
    df = pd.read_csv(data_file, encoding='utf-8')
    df['datetime'] = pd.to_datetime(df['datetime'])
    print(f"✅ Data loaded: {len(df):,} records")
    print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")
else:
    # Create synthetic data for demonstration
    print("Creating synthetic data for demonstration...")
    
    # Generate hourly data for April-August 2025
    dates = pd.date_range('2025-04-01', '2025-08-31 23:50:00', freq='10min')
    n_sensors = 5
    
    # Create realistic temperature and humidity patterns
    np.random.seed(42)
    
    # Base temperature pattern (daily cycle + seasonal trend)
    hour_of_day = dates.hour + dates.minute/60
    day_of_year = dates.dayofyear
    
    # Temperature: warmer in summer, daily cycle
    base_temp = 20 + 10 * np.sin((day_of_year - 90) * np.pi / 180)  # Seasonal
    daily_cycle = 5 * np.sin((hour_of_day - 6) * np.pi / 12)  # Daily
    temperature = base_temp + daily_cycle + np.random.normal(0, 2, len(dates))
    
    # Humidity: inverse relationship with temperature
    humidity = 70 - 0.5 * (temperature - 20) + np.random.normal(0, 10, len(dates))
    humidity = np.clip(humidity, 20, 95)  # Keep within realistic bounds
    
    df = pd.DataFrame({
        'datetime': dates,
        'serial_number': 'S001',
        'temperature': temperature,
        'humidity': humidity
    })
    
    print(f"✅ Synthetic data created: {len(df):,} records")

# Display sample data
print("\n📊 Sample data:")
df.head()

## 4. Temperature Conversion Functions

In [ ]:
def celsius_to_fahrenheit(celsius):
    """
    Convert temperature from Celsius to Fahrenheit.
    
    Parameters:
    -----------
    celsius : float or array-like
        Temperature in Celsius
    
    Returns:
    --------
    float or array-like
        Temperature in Fahrenheit
    """
    return (celsius * 9/5) + 32

def fahrenheit_to_celsius(fahrenheit):
    """
    Convert temperature from Fahrenheit to Celsius.
    
    Parameters:
    -----------
    fahrenheit : float or array-like
        Temperature in Fahrenheit
    
    Returns:
    --------
    float or array-like
        Temperature in Celsius
    """
    return (fahrenheit - 32) * 5/9

# Test conversions
test_temps_c = [0, 20, 30, 40]
test_temps_f = [celsius_to_fahrenheit(t) for t in test_temps_c]

print("Temperature Conversion Test:")
print("="*40)
for c, f in zip(test_temps_c, test_temps_f):
    print(f"{c:5.1f}°C = {f:5.1f}°F")

# Verify reverse conversion
print("\nReverse conversion check:")
for f in test_temps_f:
    c = fahrenheit_to_celsius(f)
    print(f"{f:5.1f}°F = {c:5.1f}°C")

## 5. KMA Heat Index Formula Implementation

### 5.1 The KMA Formula with Stull's Wet-bulb Temperature

The Korea Meteorological Administration (KMA) uses a formula that incorporates Stull's wet-bulb temperature estimation:

**Step 1: Calculate Wet-bulb Temperature (Tw) using Stull's formula:**

$$T_w = T_a \times \arctan(0.151977 \times (RH + 8.313659)^{0.5}) + \arctan(T_a + RH) - \arctan(RH - 1.67633) + 0.00391838 \times RH^{1.5} \times \arctan(0.023101 \times RH) - 4.686035$$

**Step 2: Calculate Heat Index (HI) using KMA formula:**

$$HI = -0.2442 + 0.55399 \times T_w + 0.45535 \times T_a - 0.0022 \times T_w^2 + 0.00278 \times T_w \times T_a + 3.0$$

Where:
- $HI$ = Heat Index (in °C)
- $T_a$ = Air Temperature (in °C)
- $T_w$ = Wet-bulb Temperature (in °C)
- $RH$ = Relative Humidity (in %)

In [ ]:
def calculate_wet_bulb_temperature(Ta, RH):
    """
    Calculate wet-bulb temperature using Stull's formula.
    
    Parameters:
    -----------
    Ta : float or array-like
        Air temperature in Celsius
    RH : float or array-like
        Relative Humidity in percent
    
    Returns:
    --------
    float or array-like
        Wet-bulb temperature in Celsius
    """
    # Stull's wet-bulb temperature formula
    Tw = (Ta * np.arctan(0.151977 * (RH + 8.313659)**0.5) + 
          np.arctan(Ta + RH) - 
          np.arctan(RH - 1.67633) + 
          0.00391838 * RH**1.5 * np.arctan(0.023101 * RH) - 
          4.686035)
    
    return Tw

def calculate_heat_index_kma(Ta, RH):
    """
    Calculate Heat Index using KMA (Korea Meteorological Administration) formula.
    
    Parameters:
    -----------
    Ta : float or array-like
        Air temperature in Celsius
    RH : float or array-like
        Relative Humidity in percent
    
    Returns:
    --------
    float or array-like
        Heat Index in Celsius
    """
    # Step 1: Calculate wet-bulb temperature
    Tw = calculate_wet_bulb_temperature(Ta, RH)
    
    # Step 2: Calculate heat index using KMA formula
    HI = (-0.2442 + 
          0.55399 * Tw + 
          0.45535 * Ta - 
          0.0022 * Tw**2 + 
          0.00278 * Tw * Ta + 
          3.0)
    
    return HI

# Alias for backward compatibility
calculate_heat_index_celsius = calculate_heat_index_kma

# Test the KMA formula
test_temps = [25, 30, 35, 40]  # Celsius
test_humidity = [40, 60, 80]

print("KMA Heat Index Calculation Test:")
print("="*60)
print("Temp(°C) | RH(%) | Tw(°C) | Heat Index(°C) | Category")
print("-"*60)

for temp in test_temps:
    for rh in test_humidity:
        tw = calculate_wet_bulb_temperature(temp, rh)
        hi = calculate_heat_index_kma(temp, rh)
        
        # Determine category based on KMA standards
        if hi < 25:
            category = "Comfortable"
        elif hi < 28:
            category = "Caution"
        elif hi < 31:
            category = "Extreme Caution"
        elif hi < 35:
            category = "Danger"
        else:
            category = "Extreme Danger"
        
        print(f"{temp:7.1f} | {rh:5.0f} | {tw:6.1f} | {hi:14.1f} | {category}")

# Comparison with simple temperature
print("\n" + "="*60)
print("Comparison: Temperature vs Heat Index (at 70% humidity)")
print("-"*60)
temps = np.arange(20, 41, 2)
humidity = 70
heat_indices = calculate_heat_index_kma(temps, humidity)

for t, hi in zip(temps, heat_indices):
    diff = hi - t
    print(f"Temp: {t:4.0f}°C → Heat Index: {hi:5.1f}°C (Difference: {diff:+5.1f}°C)")

## 6. Apply Heat Index to S-DoT Data

In [ ]:
# Calculate Heat Index for all data points using KMA formula
print("Calculating KMA Heat Index for S-DoT data...")

# Ensure we have temperature and humidity columns
if 'temperature' in df.columns and 'humidity' in df.columns:
    # Calculate wet-bulb temperature first
    df['wet_bulb_temp'] = calculate_wet_bulb_temperature(
        df['temperature'].values, 
        df['humidity'].values
    )
    
    # Calculate KMA heat index
    df['heat_index'] = calculate_heat_index_kma(
        df['temperature'].values, 
        df['humidity'].values
    )
    
    print(f"✅ KMA Heat Index calculated for {len(df):,} records")
    
    # Basic statistics
    print("\n📊 Heat Index Statistics (KMA Formula):")
    print(f"Mean: {df['heat_index'].mean():.2f}°C")
    print(f"Median: {df['heat_index'].median():.2f}°C")
    print(f"Min: {df['heat_index'].min():.2f}°C")
    print(f"Max: {df['heat_index'].max():.2f}°C")
    print(f"Std Dev: {df['heat_index'].std():.2f}°C")
    
    print("\n📊 Wet-bulb Temperature Statistics:")
    print(f"Mean: {df['wet_bulb_temp'].mean():.2f}°C")
    print(f"Max: {df['wet_bulb_temp'].max():.2f}°C")
    print(f"Min: {df['wet_bulb_temp'].min():.2f}°C")
else:
    print("❌ Temperature or humidity columns not found")

## 7. Temporal Aggregations

### 7.1 Hourly Heat Index

In [ ]:
# Calculate hourly statistics
df['hour'] = df['datetime'].dt.hour
df['date'] = df['datetime'].dt.date
df['month'] = df['datetime'].dt.month
df['month_name'] = df['datetime'].dt.strftime('%B')

# Hourly aggregation
hourly_stats = df.groupby(['date', 'hour']).agg({
    'temperature': 'mean',
    'humidity': 'mean',
    'heat_index': ['mean', 'max', 'min']
}).round(2)

print("📊 Hourly Heat Index Statistics (sample):")
print(hourly_stats.head(10))

### 7.2 Daily Maximum Heat Index

In [ ]:
# Calculate daily maximum heat index
daily_max = df.groupby('date').agg({
    'temperature': 'max',
    'humidity': 'mean',
    'heat_index': 'max'
}).round(2)

daily_max.columns = ['max_temperature', 'avg_humidity', 'max_heat_index']
daily_max = daily_max.reset_index()
daily_max['date'] = pd.to_datetime(daily_max['date'])

print(f"📊 Daily Maximum Heat Index Statistics:")
print(f"Mean: {daily_max['max_heat_index'].mean():.2f}°C")
print(f"Highest: {daily_max['max_heat_index'].max():.2f}°C on {daily_max.loc[daily_max['max_heat_index'].idxmax(), 'date'].strftime('%Y-%m-%d')}")
print(f"Lowest: {daily_max['max_heat_index'].min():.2f}°C on {daily_max.loc[daily_max['max_heat_index'].idxmin(), 'date'].strftime('%Y-%m-%d')}")

# Display top 10 hottest days
print("\n🔥 Top 10 Hottest Days by Heat Index:")
top_10_days = daily_max.nlargest(10, 'max_heat_index')[['date', 'max_temperature', 'avg_humidity', 'max_heat_index']]
print(top_10_days.to_string(index=False))

### 7.3 Monthly Patterns

In [ ]:
# Monthly statistics
monthly_stats = df.groupby('month_name').agg({
    'temperature': ['mean', 'max'],
    'humidity': 'mean',
    'heat_index': ['mean', 'max', 'min']
}).round(2)

# Reorder months
month_order = ['April', 'May', 'June', 'July', 'August']
monthly_stats = monthly_stats.reindex([m for m in month_order if m in monthly_stats.index])

print("📊 Monthly Heat Index Statistics:")
print(monthly_stats)

## 8. Visualizations

### 8.1 Heat Index Distribution

In [ ]:
# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Heat Index Distribution
axes[0, 0].hist(df['heat_index'], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0, 0].axvline(df['heat_index'].mean(), color='red', linestyle='--', label=f'Mean: {df["heat_index"].mean():.1f}°C')
axes[0, 0].set_xlabel('Heat Index (°C)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Heat Index Distribution')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Temperature vs Heat Index
scatter = axes[0, 1].scatter(df['temperature'], df['heat_index'], 
                            c=df['humidity'], cmap='YlOrRd', alpha=0.5, s=1)
axes[0, 1].plot([df['temperature'].min(), df['temperature'].max()], 
               [df['temperature'].min(), df['temperature'].max()], 
               'k--', alpha=0.5, label='T = HI')
axes[0, 1].set_xlabel('Temperature (°C)')
axes[0, 1].set_ylabel('Heat Index (°C)')
axes[0, 1].set_title('Temperature vs Heat Index (colored by humidity)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[0, 1], label='Humidity (%)')

# 3. Monthly Box Plot
monthly_data = [df[df['month_name'] == month]['heat_index'].dropna() 
                for month in month_order if month in df['month_name'].unique()]
bp = axes[1, 0].boxplot(monthly_data, labels=[m for m in month_order if m in df['month_name'].unique()])
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Heat Index (°C)')
axes[1, 0].set_title('Monthly Heat Index Distribution')
axes[1, 0].grid(True, alpha=0.3)

# 4. Hourly Pattern
hourly_pattern = df.groupby('hour')['heat_index'].mean()
axes[1, 1].plot(hourly_pattern.index, hourly_pattern.values, marker='o', linewidth=2, markersize=8)
axes[1, 1].set_xlabel('Hour of Day')
axes[1, 1].set_ylabel('Average Heat Index (°C)')
axes[1, 1].set_title('Average Daily Heat Index Pattern')
axes[1, 1].set_xticks(range(0, 24, 3))
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Heat Index Analysis', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### 8.2 Time Series Visualization

In [ ]:
# Create interactive time series plot using Plotly
fig = go.Figure()

# Add temperature trace
fig.add_trace(go.Scatter(
    x=daily_max['date'],
    y=daily_max['max_temperature'],
    mode='lines',
    name='Max Temperature',
    line=dict(color='blue', width=1),
    opacity=0.7
))

# Add heat index trace
fig.add_trace(go.Scatter(
    x=daily_max['date'],
    y=daily_max['max_heat_index'],
    mode='lines',
    name='Max Heat Index',
    line=dict(color='red', width=2)
))

# Add danger threshold line
fig.add_hline(y=41, line_dash="dash", line_color="orange", 
              annotation_text="Danger Threshold (41°C)")

# Update layout
fig.update_layout(
    title='Daily Maximum Temperature and Heat Index (April-August 2025)',
    xaxis_title='Date',
    yaxis_title='Temperature (°C)',
    hovermode='x unified',
    height=500,
    showlegend=True,
    template='plotly_white'
)

fig.show()

## 9. Heatwave Event Identification

In [ ]:
def identify_heatwave_events(daily_data, hi_threshold=35, duration_threshold=2):
    """
    Identify heatwave events based on heat index threshold and duration.
    
    Parameters:
    -----------
    daily_data : pd.DataFrame
        Daily maximum heat index data
    hi_threshold : float
        Heat index threshold for heatwave (default: 35°C)
    duration_threshold : int
        Minimum consecutive days for heatwave (default: 2 days)
    
    Returns:
    --------
    pd.DataFrame
        Heatwave events with start date, end date, and duration
    """
    # Mark days exceeding threshold
    daily_data['is_hot'] = daily_data['max_heat_index'] >= hi_threshold
    
    # Find consecutive hot days
    daily_data['group'] = (daily_data['is_hot'] != daily_data['is_hot'].shift()).cumsum()
    
    # Identify heatwave events
    heatwaves = []
    
    for group_id, group_data in daily_data[daily_data['is_hot']].groupby('group'):
        duration = len(group_data)
        if duration >= duration_threshold:
            heatwaves.append({
                'start_date': group_data['date'].min(),
                'end_date': group_data['date'].max(),
                'duration_days': duration,
                'max_heat_index': group_data['max_heat_index'].max(),
                'avg_heat_index': group_data['max_heat_index'].mean()
            })
    
    return pd.DataFrame(heatwaves)

# Identify heatwave events
heatwave_events = identify_heatwave_events(daily_max.copy())

if len(heatwave_events) > 0:
    print(f"🌡️ Identified {len(heatwave_events)} heatwave events:")
    print("="*70)
    print(heatwave_events.to_string(index=False))
    
    # Summary statistics
    print("\n📊 Heatwave Statistics:")
    print(f"Total heatwave days: {heatwave_events['duration_days'].sum()}")
    print(f"Average duration: {heatwave_events['duration_days'].mean():.1f} days")
    print(f"Longest heatwave: {heatwave_events['duration_days'].max()} days")
    print(f"Highest heat index during heatwave: {heatwave_events['max_heat_index'].max():.1f}°C")
else:
    print("No heatwave events identified with current thresholds")

## 10. Save Processed Data with Heat Index

In [ ]:
# Save data with heat index
output_file = os.path.join(DATA_PATH, 'sdot_with_heat_index.csv')

# Select columns to save
columns_to_save = ['datetime', 'serial_number', 'temperature', 'humidity', 'heat_index']
save_columns = [col for col in columns_to_save if col in df.columns]

try:
    df[save_columns].to_csv(output_file, index=False, encoding='utf-8')
    print(f"✅ Data with Heat Index saved successfully!")
    print(f"📁 File: {output_file}")
    
    # Save daily summary
    daily_output = os.path.join(DATA_PATH, 'daily_heat_index_summary.csv')
    daily_max.to_csv(daily_output, index=False, encoding='utf-8')
    print(f"📁 Daily summary saved: {daily_output}")
    
    # Save heatwave events
    if len(heatwave_events) > 0:
        heatwave_output = os.path.join(DATA_PATH, 'heatwave_events.csv')
        heatwave_events.to_csv(heatwave_output, index=False, encoding='utf-8')
        print(f"📁 Heatwave events saved: {heatwave_output}")
        
except Exception as e:
    print(f"❌ Error saving file: {e}")

## 11. Assignment

### Week 3 Tasks:

1. **KMA Heat Index Implementation** (30 points)
   - Implement Stull's wet-bulb temperature calculation
   - Implement the KMA Heat Index formula
   - Verify calculations with test cases
   - Compare results with actual KMA data if available

2. **Data Analysis** (30 points)
   - Calculate heat index for all S-DoT data
   - Create daily, hourly, and monthly aggregations
   - Identify the hottest periods by heat index
   - Analyze wet-bulb temperature patterns

3. **Heatwave Analysis** (25 points)
   - Identify heatwave events using KMA thresholds
   - Calculate heatwave duration and intensity
   - Compare heatwaves across different months
   - Apply Korean heatwave warning criteria

4. **Visualization** (15 points)
   - Create at least 3 different visualizations
   - Include time series and distribution plots
   - Show relationship between temperature, humidity, and heat index
   - Add appropriate labels and legends in both English and Korean

### Bonus Challenge (10 extra points):
- Compare KMA formula with NOAA formula and analyze differences
- Create an interactive dashboard showing real-time heat index categories
- Analyze the relationship between heat index and air quality (PM2.5/PM10)
- Implement heat wave warning system based on KMA criteria

### Submission:
- Complete notebook saved as `Week03_YourName.ipynb`
- Include processed data with KMA heat index
- Document your findings and insights
- Include comparison with temperature-only analysis

## 12. Summary

In this week, we covered:
- ✅ Understanding Heat Index and its importance
- ✅ Temperature conversion between Celsius and Fahrenheit
- ✅ Implementation of NOAA Heat Index formula
- ✅ Calculation of heat index for S-DoT data
- ✅ Temporal aggregations (hourly, daily, monthly)
- ✅ Heatwave event identification
- ✅ Comprehensive visualizations

### Key Findings:
1. **Heat Index vs Temperature**: Heat index can be significantly higher than actual temperature during humid conditions
2. **Daily Patterns**: Heat index typically peaks in mid-afternoon (2-4 PM)
3. **Seasonal Trends**: July and August show the highest heat index values
4. **Heatwave Events**: Multiple multi-day heatwave events identified

### Important Formulas:
- **Celsius to Fahrenheit**: °F = (°C × 9/5) + 32
- **NOAA Heat Index**: Complex polynomial equation with 9 terms
- **Heat Danger Threshold**: 41°C (105°F)

### Next Week Preview:
**Week 4: Spatial Analysis & GIS**
- Load Seoul administrative boundaries
- Map sensor locations
- Create heat maps
- Identify spatial patterns
- Interactive map visualization

---
**End of Week 3**

*Instructor: Sohn Chul*